# Dados a serem coletados

- Nome -> Steam - Ok
- AppID -> http://api.steampowered.com/ISteamApps/GetAppList/v2/
- Preço -> Steam - Ok

- Quantidade de avaliação -> Steam - OK
- Lançamento -> Steam - OK
- Idade recomendada -> Steam - 1/2 Ok (parece que alguns jogos não tem essa info)

- Duração do jogo -> How Long To beat

- Marcadores/Tags -> SteamSpy
- Avaliação dos usuário -> SteamSpy

- Free - Steam - Ok
---
- Acesso antecipado - Steam -> É um gênero, podendo ser filtrado por isso
- DLC - Steam -Ok
- Lançamento futuro - Steam -> 'release_date': {'coming_soon': True, 'date': 'Q1 2024'}
- Línguas -> Steam - Ok
- Características do jogo -> Steam - Ok
- Publicadora -> Steam - Ok
- Desenvolvedora -> Steam - OK
- Gênero -> Steam - Ok
- Conquistas (quantidade) -> Steam - OK

- Item na loja (quantidade) -> Steam ?

- Avaliação Crítica -> Metacritic, OpenCritic, Steam - 1/2 OK (Só tem metacritic pelo steam) 
- Requisitos Mínimos (Windows) -> Steam - Ok
- Requisitos Máximos (Windows) -> Steam - OK

# HTMLs

- Características: class="game_area_features_list_ctn">


### Coleta de dados por jogo

In [31]:
import requests as rs
import json

url = "http://store.steampowered.com/api/appdetails/"

appIds = []

appIdTest = 435150
dlcAppId = 715950 #-> Testar se conseguimos diferenciar DLC de jogo

currency='us'
language='en'

params = {"appids": appIdTest, "cc": currency, "l": language}

response = rs.get(url=url, params=params, timeout=10)
data = response.json()['435150']['data']
for a in data:
    print(a)
#print(response.json()['435150']['data']['price_overview'])



type
name
steam_appid
required_age
is_free
controller_support
dlc
detailed_description
about_the_game
short_description
supported_languages
reviews
header_image
capsule_image
capsule_imagev5
website
pc_requirements
mac_requirements
linux_requirements
legal_notice
developers
publishers
price_overview
packages
package_groups
platforms
metacritic
categories
genres
screenshots
movies
recommendations
achievements
release_date
support_info
background
background_raw
content_descriptors


### Coleta de AppIds

In [29]:
import requests as rs
import json

url_AppIds = "http://api.steampowered.com/ISteamApps/GetAppList/v2/"

response = rs.get(url=url_AppIds,timeout = 10)
if response:
    data = response.json()
    appIds = data['applist']['apps'] #Apps é uma lista de dicionários, com chaves "appid" e "name"
    with open('Steam_AppIds.json','w') as file:
        json.dump(appIds,file,indent=4)
    print(f'Arquivo de appIDs foi criado com {len(appIds)} IDs')
else:
    print('A API não retornou resposta')

Arquivo de appIDs foi criado com 179186 IDs


In [38]:
import json

appIds = []

with open('Steam_AppIds.json','r') as file:
    appIds = json.load(file)

appIdsRange = int(len(appIds)/100)

print(appIdsRange)

currency='us'
language='en'

count = 0
for s in range(0,100): # Testar com apenas 1 segmento primeiro
    if s == 99:
        print(len(appIds[appIdsRange*s:len(appIds)]))
        count += len(appIds[appIdsRange*s:len(appIds)])
    else:
        count += len(appIds[appIdsRange*s:appIdsRange*(s+1)])
print (count == len(appIds))

1791
1877
True


### Funções úteis

In [4]:
import requests as rs
import time

columnsToIgnore =['dlc', 'detailed_description', 'about_the_game', 'reviews', 'header_image', 'capsule_image', 'capsule_imagev5', 'website' , 'mac_requirements', 
                  'linux_requirements', 'legal_notice', 'packages', 'package_groups', 'screenshots', 'movies', 'support_info', 'background', 'background_raw', 'content_descriptors']

def RemoveUselessData(data):
    for k in columnsToIgnore: 
        try:
            data.pop(k)
        except:
            continue

currency='us'
language='en'
#steam_appid trocar os appid por steam_appid para manter um padrão... ou organizar isso quando acabar já q já foi tão longe
def GetData(appid,name,count=0):
    url = "http://store.steampowered.com/api/appdetails/"
    params = {"appids": appid, "cc": currency, "l": language}
    try:
        response = rs.get(url=url, params=params, timeout=10)
        if response:
            try:
                data = response.json()[str(appid)]['data'] # => É um dicionário
                for k in columnsToIgnore: 
                    try:
                        data.pop(k)
                    except:
                        continue
                data['scrap_status'] = 'Scrap_Sucess'
            except KeyError:
                data = {'appid':appid,'name':name,'scrap_status':'Scrap_NoData'}
        elif response.status_code == 429:
            print(f'Esperando api, requisição {count}')
            time.sleep(300) # => A API da steam só aceita N solicitações a cada 5 minutos
            data = GetData(appid,name)
        else:
            data = {'appid':appid,'name':name,'scrap_status': f'Scrap_Fail {response}'}
    except Exception as e:
        data = {'appid':appid,'name':name,'scrap_status':f'Scrap_Fail {type(e)}'}
        print(f'{appid} - Erro: {e}')
    return data

### Scrap em segmentos/partes de toda a Steam

In [2]:
#import requests as rs
import json
from tqdm.notebook import trange, tqdm
import os

appIds = []

with open('Steam_AppIds.json','r') as file:
    appIds = json.load(file)

appIdsRange = int(len(appIds)/100)

for s in range(0,100):
    if os.path.exists(f'RawData/SteamAppData_{s}.json'):
        #print(f'Pacote de dados {s} já extraído')
        continue
    else:
        if s == 99:
            segment = appIds[appIdsRange*s:len(appIds)]
        else:
            segment = appIds[appIdsRange*s:appIdsRange*(s+1)]
        dataList = []
        progresso = tqdm(total=1, desc=f"Progresso do {s}º segmento", position=0, leave=True)
        for n, app in enumerate(segment):
            dataList.append(GetData(app['appid'],app['name'],n))
            progresso.update(1/len(segment))
        with open(f'RawData/SteamAppData_{s}.json','w') as file:
            json.dump(dataList,file,indent=4)
        print(f'Pacotes de dados coletados: {s}/99')

### Tentando pegar os dados denovo das coletas que falharam

In [5]:
import json
from tqdm.notebook import tqdm

def ReScrap(tryNoData = False):
    with open('Datasets/SteamDataset.json','r') as file:
        data = json.load(file)

    noDataCount = 0
    progress = tqdm(total=1, desc=f"Progresso:", position=0, leave=True)

    for i,app in enumerate(data):##list(filter(lambda a: 'Scrap_Sucess' not in a.get('scrap_status'), data)):
        status = app['scrap_status']
        if "Scrap_Fail" in status:
            data[i] = GetData(app['appid'],app['name'])
        elif tryNoData and 'Scrap_NoData' in status:
            data[i] = GetData(app['appid'],app['name'])
            if data[i]['scrap_status'] == 'Scrap_Sucess':
                appId = data[i]['steam_appid']
                appName = data[i]['name']
                print(f'Dados encontrados! {appId} - {appName}')
            else:
                noDataCount+=1
        progress.update(1/len(data))

    print(noDataCount)
    with open('Datasets/SteamDataset.json','w') as file:
        json.dump(data,file,indent=4)

ReScrap()

Progresso::   0%|          | 0/1 [00:00<?, ?it/s]

1863540 - Erro: Expecting value: line 1 column 1 (char 0)
1444140 - Erro: Expecting value: line 1 column 1 (char 0)
1061400 - Erro: Expecting value: line 1 column 1 (char 0)
2124470 - Erro: Expecting value: line 1 column 1 (char 0)
0


### Unificador de Jsons

In [10]:
import json
from tqdm.notebook import tqdm

steamscrap = []
progresso = tqdm(total=1, desc="Progresso: ", position=0, leave=True)

for i in range(0,100):
    with open(f'HowLongToBeatDataAdded/SteamAppData_{i}.json','r') as file:
        if i == 0:
            steamscrap = json.load(file)
        else:
            steamscrap += json.load(file)
        progresso.update(0.01)

with open('Datasets/SteamDataset.json','w') as file:
    json.dump(steamscrap,file,indent=4)

Progresso:   0%|          | 0/1 [00:00<?, ?it/s]

dict_keys(['success'])


### Adicionar novos apps

Motivo de existência: Para atualizar o dataset, provavelmente não vai fazer sentido coletar dados de DLCs e etc. dessa forma, primeiro é necessário adicionar os novos APPs para poder filtrar isso e só pegar os dados que realmente interessam, a fim de reduzir substancialmente o tempo para cada coleta de dados

1. Atualizar a lista de APPIDs
2. Ler o arquivo gerado e subtrair os APPs já conhecidos pelo dataset
3. Coletar os dados dos novos APPs e anexar ao dataset

Alguns jogos não tem nenhum tipo de retorno da API, exemplo 1594690, nesse caso é um jogo VR, será que tem relação?
No caso do 1594890 era necessário realizar login, pois era conteúdo sexual
No caso do 2472540 a página da loja nem existe

ShowAppTagModal( 1982240 ) => Evento OnClick que exibe todas as tags de um jogo na página steam, pode ser usado para pegar tags faltantes

As tags parecem ser armazenadas em rgAppTags

### Local onde todas as tags estão na página Steam

//*[@id="responsive_page_template_content"]/script[5]

/html/body/div[1]/div[7]/div[6]/script[5]

<script type="text/javascript">
		$J( function() {
			InitAppTagModal( 1982240,
				[{"tagid":122,"name":"RPG","count":64,"browseable":true},{"tagid":4231,"name":"RPG de A\u00e7\u00e3o","count":37,"browseable":true},{"tagid":4474,"name":"CRPG","count":35,"browseable":true},{"tagid":4191,"name":"3D","count":32,"browseable":true},{"tagid":4295,"name":"Futurista","count":29,"browseable":true},{"tagid":3993,"name":"Combate","count":22,"browseable":true},{"tagid":4182,"name":"Um Jogador","count":21,"browseable":true}],
				[],
				"1_5_9__410",
				"1_5_9__411",
				null,
				false			);

						if ( typeof GDynamicStore != 'undefined' )
				GDynamicStore.FixupNamePortion();
			
					});
	</script>

É possível coletar as tags da página steam procurando InitAppTagModal na front Page